In [ ]:
from google.colab import drive
import sys
import os

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

drive.mount("/content/gdrive")
#Specify path path of the directory
#path = "/content/gdrive/My Drive/Colab Notebooks/econ_indicator"
#sys.path.append(path)
os.chdir('/content/gdrive/My Drive/Colab Notebooks/econ_indicator')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import IPython
from google.colab import output
display(IPython.display.Javascript('''
function ClickConnect(){
  btn=document.querySelector("colab-connect-button")
  if(btn != null){
    console.log("Click colab-connect-button");
    btn.click()
  }
  btn=document.getElementById('ok')
  if(btn != null){
    console.log("Click reconnect");
    btn.click()
  }
}
setInterval(ClickConnect,60000)
'''))

<IPython.core.display.Javascript object>

In [ ]:
# !pwd
!ls

 All_Data_Formatted.xlsx			       loss1.csv
 basicmodels_2.ipynb				       loss1.xlsx
 breakeven_inflation10.csv			       loss2.xlsx
 check.csv					       loss3.xlsx
 consumer_price_index.csv			       loss_4.csv
 dailysalesexample.ipynb			       loss.csv
 DiscreteDerivativeWithCustomizeableTimeDelays.ipynb   loss_temp2.csv
'econ 2nd try.ipynb'				       loss_temp.csv
 employ_rate.csv				       loss_test.csv
 FRED_Data.ipynb				       mortgage_rate30.csv
 fredgraph.xls					       neural_network.ipynb
 industrial_production.csv			       predicted_sales.csv
 loss012.csv					       ProphetDerivative.ipynb
 loss_0.csv					       sales_indicators.csv
 loss_12.csv					       temp1.csv
 loss1_5_2.csv					       temp.csv
 loss1_5.csv






*   change epochs, batch size;
*   adding 1/8 layer, change drop out rate
*  change x,y








In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
#df = pd.read_csv('sales_indicators.csv',index_col=0)
df = pd.read_excel('All_Data_Formatted.xlsx', skiprows=1,index_col=0)
df['International_Trade']=df['International_Trade'].div(10**8)
df.reset_index(inplace=True)
df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
df['Holiday'] = df['month'].isin([12])
df.Holiday.sum()

19

In [ ]:
df.replace({False: 0, True: 1}, inplace=True)
df

,date,RSAFS,RSFSXMV,RSHPCS,RSGASS,RSCCAS,RSSGHBMS,RSGMS,RSDSELD,RSMSR,...,employ_rate,indust_prod,mortgage_rate30,Sticky_Price_Cpi,International_Trade,Med_Cpi,CPI,year,month,Holiday
0,2003-01-01,295248,225575,15441,23073,14565,5794,37740,17807,8565,...,71.590301,91.2395,5.9160,2.787553,974.91,2.181390,76.661168,2003,1,0
1,2003-02-01,291167,224632,15534,23885,14142,5576,38030,17670,8372,...,71.520526,91.3384,5.8425,2.704659,931.54,1.975830,77.251843,2003,2,0
2,2003-03-01,296325,227697,15677,23968,14465,5660,38391,17763,8405,...,71.396225,91.0690,5.7450,2.627787,1058.42,1.126212,77.715944,2003,3,0
3,2003-04-01,295600,226020,15705,22581,14434,5709,38321,17687,8359,...,71.378556,90.4913,5.8125,2.459349,1038.69,0.553037,77.547180,2003,4,0
4,2003-05-01,296410,226750,15791,21810,14651,5748,38483,17838,8525,...,71.213061,90.4697,5.4840,2.348341,1020.68,1.075788,77.420607,2003,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2021-12-01,634393,513586,32797,55910,25051,8968,67197,10253,14285,...,70.799610,101.7587,3.0980,3.467011,2578.86,5.275819,117.629537,2021,12,1
228,2022-01-01,651557,521891,32815,54919,25160,8877,69233,11376,14527,...,70.748899,102.1460,3.4450,3.982988,2479.92,7.141525,118.619340,2022,1,0
229,2022-02-01,662321,530893,32445,58392,25481,8916,67568,11397,14986,...,70.890307,102.9981,3.7625,4.297858,2349.21,6.506399,119.702806,2022,2,0
230,2022-03-01,669958,541436,32763,64656,26157,9009,68468,11393,15490,...,71.311622,103.7286,4.1720,4.551815,2965.18,5.902594,121.301004,2022,3,0


In [ ]:
df.set_index('date', inplace=True)
df

,RSAFS,RSFSXMV,RSHPCS,RSGASS,RSCCAS,RSSGHBMS,RSGMS,RSDSELD,RSMSR,RSNSR,...,employ_rate,indust_prod,mortgage_rate30,Sticky_Price_Cpi,International_Trade,Med_Cpi,CPI,year,month,Holiday
date,,,,,,,,,,,,,,,,,,,,,
2003-01-01,295248,225575,15441,23073,14565,5794,37740,17807,8565,16434,...,71.590301,91.2395,5.9160,2.787553,974.91,2.181390,76.661168,2003,1,0
2003-02-01,291167,224632,15534,23885,14142,5576,38030,17670,8372,16933,...,71.520526,91.3384,5.8425,2.704659,931.54,1.975830,77.251843,2003,2,0
2003-03-01,296325,227697,15677,23968,14465,5660,38391,17763,8405,16764,...,71.396225,91.0690,5.7450,2.627787,1058.42,1.126212,77.715944,2003,3,0
2003-04-01,295600,226020,15705,22581,14434,5709,38321,17687,8359,16593,...,71.378556,90.4913,5.8125,2.459349,1038.69,0.553037,77.547180,2003,4,0
2003-05-01,296410,226750,15791,21810,14651,5748,38483,17838,8525,16722,...,71.213061,90.4697,5.4840,2.348341,1020.68,1.075788,77.420607,2003,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,634393,513586,32797,55910,25051,8968,67197,10253,14285,95958,...,70.799610,101.7587,3.0980,3.467011,2578.86,5.275819,117.629537,2021,12,1
2022-01-01,651557,521891,32815,54919,25160,8877,69233,11376,14527,102773,...,70.748899,102.1460,3.4450,3.982988,2479.92,7.141525,118.619340,2022,1,0
2022-02-01,662321,530893,32445,58392,25481,8916,67568,11397,14986,104095,...,70.890307,102.9981,3.7625,4.297858,2349.21,6.506399,119.702806,2022,2,0


In [ ]:
df.columns

Index(['RSAFS', 'RSFSXMV', 'RSHPCS', 'RSGASS', 'RSCCAS', 'RSSGHBMS', 'RSGMS',
       'RSDSELD', 'RSMSR', 'RSNSR', 'RSFSDP', 'RSXFS', 'break_inflate10',
       'employ_rate', 'indust_prod', 'mortgage_rate30', 'Sticky_Price_Cpi',
       'International_Trade', 'Med_Cpi', 'CPI', 'year', 'month', 'Holiday'],
      dtype='object')

In [ ]:
df.describe()

,RSAFS,RSFSXMV,RSHPCS,RSGASS,RSCCAS,RSSGHBMS,RSGMS,RSDSELD,RSMSR,RSNSR,...,employ_rate,indust_prod,mortgage_rate30,Sticky_Price_Cpi,International_Trade,Med_Cpi,CPI,year,month,Holiday
count,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,...,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000,232.000000
mean,421030.331897,337520.357759,23446.137931,38962.172414,19452.409483,6825.336207,52987.616379,14553.724138,9810.831897,41111.176724,...,69.433739,97.604345,4.639767,2.267598,1763.719353,2.475998,96.086533,2012.172414,6.431034,0.081897
std,86868.500546,69949.773485,4624.455444,7652.069392,3011.910132,741.210312,7231.968193,2602.261231,1398.675339,22340.382446,...,2.059045,4.604815,1.097578,0.600239,361.318510,1.085911,10.541555,5.595725,3.472151,0.274800
min,291167.000000,224632.000000,15441.000000,21777.000000,2866.000000,3693.000000,37740.000000,6269.000000,7179.000000,16434.000000,...,60.261014,84.727700,2.684000,0.667614,931.540000,-0.286064,76.661168,2003.000000,1.000000,0.000000
25%,355459.750000,285424.250000,20007.500000,34863.750000,17397.000000,6469.750000,48770.000000,12319.000000,8900.750000,25295.000000,...,67.527579,94.567025,3.789500,2.023419,1538.207500,1.936043,88.500403,2007.000000,3.000000,0.000000
50%,403871.000000,329697.000000,22911.500000,39293.000000,19552.000000,6721.500000,53605.000000,14716.000000,9483.500000,34594.500000,...,69.829709,98.782450,4.398750,2.275491,1835.180000,2.378864,97.104005,2012.000000,6.000000,0.000000
75%,473641.750000,375053.000000,26853.000000,44579.250000,21628.250000,7015.000000,57620.000000,17268.000000,10201.750000,50342.750000,...,71.254773,101.313500,5.733750,2.569466,1982.325000,2.957473,103.295650,2017.000000,9.000000,0.000000
max,674719.000000,543908.000000,32962.000000,64656.000000,26166.000000,9513.000000,69549.000000,18263.000000,15864.000000,104095.000000,...,72.323954,104.522400,6.762500,4.723538,2965.180000,7.141525,121.978170,2022.000000,12.000000,1.000000


In [ ]:
# #break time periods
# #2010-2019 June
# #2019 July-2019 Dec
# df_2010_2019June=df.iloc[84:198,:]
# df_2010_2019June

# df_2010_2018Dec=df.iloc[84:192,:]
# df_2010_2018Dec

df_2010_2019=df.iloc[84:204,:]
df_2010_2019

,RSAFS,RSFSXMV,RSHPCS,RSGASS,RSCCAS,RSSGHBMS,RSGMS,RSDSELD,RSMSR,RSNSR,...,employ_rate,indust_prod,mortgage_rate30,Sticky_Price_Cpi,International_Trade,Med_Cpi,CPI,year,month,Holiday
date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,346043,289245,21187,37399,17351,6507,49844,15585,8363,27236,...,66.750593,89.1936,5.0300,1.177002,1367.25,0.680111,91.422556,2010,1,0
2010-02-01,346835,291032,21199,36950,17402,6412,50538,15507,8251,27249,...,66.709214,89.5089,4.9900,1.054847,1338.98,-0.106628,91.445339,2010,2,0
2010-03-01,354360,293463,21363,37123,17908,6515,50095,15479,8705,27852,...,66.732157,90.1449,4.9675,0.931839,1577.28,-0.286064,91.820840,2010,3,0
2010-04-01,357461,295638,21471,37218,17724,6409,50650,15744,8567,27767,...,66.868325,90.4740,5.0980,0.876030,1531.63,0.452616,91.980322,2010,4,0
2010-05-01,354255,292684,21399,36606,17656,6496,50023,15399,8632,27752,...,66.755470,91.7131,4.8875,0.799868,1561.24,0.275748,92.051625,2010,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,521158,415426,28580,42735,22615,6740,60074,11369,11045,63727,...,71.435858,102.9086,3.6160,2.633395,2153.85,2.587546,108.244556,2019,8,0
2019-09-01,517755,414270,28600,42589,22376,6698,59377,11042,10997,63995,...,71.722503,102.5790,3.6050,2.616559,2063.62,2.820512,108.329360,2019,9,0
2019-10-01,519519,416179,28809,43254,22417,6742,59662,10969,11156,64346,...,71.687213,101.7775,3.6880,2.734200,2191.58,2.360034,108.577022,2019,10,0


In [ ]:
# df_2019July_Dec=df.iloc[198:204,:]
# df_2019July_Dec

# df_2019=df.iloc[192:204,:]
# df_2019

In [ ]:
# df.head()

In [ ]:
row_num=df_2010_2019.shape[0]

In [ ]:
##################################################testing function 
def neural_net(department,econ_id,ep,bat):
  Y = df_2010_2019.loc[:,department] 
  x_temp=['year','month']+econ_id
  # x_temp=['year','month']
  X = df_2010_2019.loc[:,x_temp]
  #X = df_2010_2018Dec.loc[:,['year','month']]

  #Split train and test
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 1/10, shuffle=True,random_state=66)

  #Count features for modelization
  X_num_columns= len(X.columns)

  # normalizer=layers.Normalizaton()
  # normalizer.adapt(X_train)

  model = Sequential()
  # model.add(normalizer)
  model.add(Dense(120, activation='relu',input_dim = X_num_columns))
  model.add(Dense(60, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(30, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1,activation='linear'))
  # model.add(Dropout(0.2))

  # model.add(Dense(30,
  #               activation='relu'))
  # model.add(Dropout(0.2))

  model.compile(optimizer='adam', loss='mean_squared_error')

  model.fit(X_train, y_train, epochs=ep, batch_size=bat)

  loss = model.evaluate(X_test, y_test)

  result=(df_2010_2019[department].std()-np.sqrt(loss))/df_2010_2019[department].std()
  result=result*100
  return result


In [ ]:
# econ_id=['indust_prod', 'mortgage_rate30']
# x_temp=['year','month']+econ_id
# print(x_temp)

In [ ]:
##########################################testing each department with single economic indicators
sales=['RSAFS', 'RSFSXMV', 'RSHPCS', 'RSGASS', 'RSCCAS', 'RSSGHBMS', 'RSGMS', 'RSDSELD', 'RSMSR', 'RSNSR', 'RSFSDP', 'RSXFS']
# sales=['RSMSR', 'RSNSR', 'RSFSDP', 'RSXFS']
econ_indicators=['break_inflate10','employ_rate','indust_prod','International_Trade', 'CPI','Holiday']
# econ_indicators=[" "]

# epoch = [64,512]
batch = 16
# batch = [16,25,32,64]

ep = 128
# ep = 64


#creating combinations of economic indicators
# comb0 = [' ']

comb1 = econ_indicators
comb1 = list(map(lambda el:[el], comb1))
# # print(len(comb1))

from itertools import combinations
comb2 = list(combinations(econ_indicators, 2))
comb2 = list(map(list, comb2))
# # print(len(comb2))

# comb3 = list(combinations(econ_indicators, 3))
# comb3 = list(map(list, comb3))
# # print(len(comb3))

# comb4 = list(combinations(econ_indicators, 4))
# comb4 = list(map(list, comb4))
# print(len(comb4))

# comb5 = [['indust_prod', 'mortgage_rate30', 'International_Trade', 'CPI','Holiday']]
# # print(len(comb5))

comb_econ_indicators = comb1+comb2
# comb_econ_indicators = comb3+comb4+comb5
# comb_econ_indicators = comb1+comb2+comb3+comb4+comb5

# def neural_net(department,econ_id,ep,bat):
#   return 1

# print(len(sales))#12
# print(len(comb_econ_indicators))#36

import numpy as np
loss = np.zeros((len(sales),len(comb_econ_indicators)))
# loss = np.zeros((len(sales)))
#print(loss)
# for department,econ_id in zip(sales, comb_econ_indicators):
#   temp = [neural_net(department,econ_id,ep,bat) for bat in batch]
#   loss.append(temp)

for i, department in enumerate(sales):
    for j, econ_id in enumerate(comb_econ_indicators):
        temp = neural_net(department,econ_id,ep,batch)
        loss[i,j]=temp

# econ_id=""
# for i, department in enumerate(sales):
#       temp = neural_net(department,econ_id,ep,batch)
#       loss[i]=temp

#department='RSAFS'
# econ_id=['indust_prod', 'mortgage_rate30']
# ep = 512
# bat = 32
#neural_net(department,econ_id,ep,bat)

# sales_indust = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_mortgage = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_intern = [neural_net(i,econ_id,ep,bat) for i in sales]
#sales_cpi = [neural_net(i,econ_id,ep,bat) for i in sales]



Epoch 1/128
7/7 [==============================] - 1s 2ms/step - loss: 195598319616.0000
Epoch 2/128
7/7 [==============================] - 0s 2ms/step - loss: 195332472832.0000
Epoch 3/128
7/7 [==============================] - 0s 2ms/step - loss: 195045834752.0000
Epoch 4/128
7/7 [==============================] - 0s 2ms/step - loss: 194587688960.0000
Epoch 5/128
7/7 [==============================] - 0s 2ms/step - loss: 194083127296.0000
Epoch 6/128
7/7 [==============================] - 0s 2ms/step - loss: 193248739328.0000
Epoch 7/128
7/7 [==============================] - 0s 2ms/step - loss: 192238288896.0000
Epoch 8/128
7/7 [==============================] - 0s 2ms/step - loss: 190799036416.0000
Epoch 9/128
7/7 [==============================] - 0s 2ms/step - loss: 188747956224.0000
Epoch 10/128
7/7 [==============================] - 0s 2ms/step - loss: 186109788160.0000
Epoch 11/128
7/7 [==============================] - 0s 2ms/step - loss: 182587572224.0000
Epoch 12/128
7/7 [=

1/1 [==============================] - 0s 97ms/step - loss: 2505953536.0000
Epoch 1/128
7/7 [==============================] - 0s 2ms/step - loss: 195438411776.0000
Epoch 2/128
7/7 [==============================] - 0s 2ms/step - loss: 195056418816.0000
Epoch 3/128
7/7 [==============================] - 0s 2ms/step - loss: 194662809600.0000
Epoch 4/128
7/7 [==============================] - 0s 2ms/step - loss: 194127970304.0000
Epoch 5/128
7/7 [==============================] - 0s 2ms/step - loss: 193454260224.0000
Epoch 6/128
7/7 [==============================] - 0s 2ms/step - loss: 192444776448.0000
Epoch 7/128
7/7 [==============================] - 0s 2ms/step - loss: 191072124928.0000
Epoch 8/128
7/7 [==============================] - 0s 2ms/step - loss: 189547036672.0000
Epoch 9/128
7/7 [==============================] - 0s 2ms/step - loss: 186939850752.0000
Epoch 10/128
7/7 [==============================] - 0s 2ms/step - loss: 183951065088.0000
Epoch 11/128
7/7 [===============

Streaming output truncated to the last 5000 lines.
7/7 [==============================] - 0s 2ms/step - loss: 3349559040.0000
Epoch 72/128
7/7 [==============================] - 0s 2ms/step - loss: 2997763840.0000
Epoch 73/128
7/7 [==============================] - 0s 4ms/step - loss: 2951223296.0000
Epoch 74/128
7/7 [==============================] - 0s 8ms/step - loss: 3356772096.0000
Epoch 75/128
7/7 [==============================] - 0s 3ms/step - loss: 3654786560.0000
Epoch 76/128
7/7 [==============================] - 0s 2ms/step - loss: 3406206720.0000
Epoch 77/128
7/7 [==============================] - 0s 2ms/step - loss: 3772570112.0000
Epoch 78/128
7/7 [==============================] - 0s 2ms/step - loss: 2583041792.0000
Epoch 79/128
7/7 [==============================] - 0s 2ms/step - loss: 3508471808.0000
Epoch 80/128
7/7 [==============================] - 0s 2ms/step - loss: 3044288512.0000
Epoch 81/128
7/7 [==============================] - 0s 2ms/step - loss: 2937005312

In [ ]:
# np.sum(loss)
# loss.shape
print(loss)
print(loss.shape)

[[-6.52517281e+00 -4.96360585e+00 -3.70304118e+00  2.56302355e+01
  -3.13562656e+00 -4.71063249e+00 -4.49212893e+00 -3.62878073e+00
   2.49824931e+01 -3.31064955e+00 -5.25684516e+00 -3.84426652e+00
   2.61104952e+01 -3.24757400e+00 -4.34494910e+00  2.75530901e+01
  -2.28624112e+00 -4.76656795e+00  2.63785541e+01  2.57692235e+01
  -3.36938723e+00]
 [-2.19153899e+00 -2.06222410e+00 -4.56618190e-01  3.20354264e+01
  -1.43661509e+00 -1.64852239e+00 -1.29478821e+00 -5.82903090e-01
   3.02842327e+01 -4.02313829e-01 -2.20435426e+00 -2.88757520e-01
   2.92478347e+01  3.21770722e-01 -2.54869796e+00  3.04553523e+01
   1.21235400e+00 -1.03844349e+00  3.10508914e+01  2.92203316e+01
  -3.23069879e-01]
 [-1.61665055e+00 -3.87676610e+00 -8.55229131e-01  3.02872372e+01
  -1.43840303e+00 -1.71679414e+00 -1.25720543e+00 -3.55101106e+00
   3.10950883e+01  3.58791728e-01 -1.27265440e+00 -1.92434344e+00
   2.95922315e+01 -1.08026667e-02 -7.16984136e-01  2.26391048e+01
   9.70124873e-01 -3.03626408e+00  2.1

In [ ]:
import pandas as pd
df_loss=pd.DataFrame(loss)
df_loss.to_csv("loss_12.csv")

In [ ]:
sales=['RSAFS', 'RSFSXMV', 'RSHPCS', 'RSGASS', 'RSCCAS', 'RSSGHBMS', 'RSGMS', 'RSDSELD', 'RSMSR', 'RSNSR', 'RSFSDP', 'RSXFS']
# econ_indicators=['indust_prod', 'mortgage_rate30', 'International_Trade', 'CPI','Holiday']

#creating combinations of economic indicators
comb0 = ['no_indicator']
comb1 = econ_indicators
# comb1 = list(map(lambda el:[el], comb1))

from itertools import combinations
comb2 = list(combinations(econ_indicators, 2))
comb2 = list(map(list, comb2))

comb2_join = []
for x in comb2:
  x_join = '+'.join(x)
  comb2_join.append(x_join)

comb_econ_indicators = comb1+comb2_join

df1 = pd.read_csv('loss_12.csv')
df1 = df1.drop(columns=['Unnamed: 0'])
# df1.shape
df1.columns = comb_econ_indicators
df1.index = sales

df0 = pd.read_csv('loss_0.csv')
df0 = df0.drop(columns=['Unnamed: 0'])
# df1.shape
df0.columns = ["no_indicator"]
df0.index = sales

df_loss012=pd.concat([df0,df1], axis=1)
df_loss012.to_csv("loss012.csv")